# Packages 

In [85]:
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import glob
import pathlib
import unicodedata
import tarfile

from pandarallel import pandarallel
pandarallel.initialize()
from unidecode import unidecode

def get_csv_size(csv_name):
    get_size = os.path.getsize(os.getcwd() + '/'+csv_name)
    mb_size = get_size/(1024 * 1024)
    mb_size = round(mb_size,1)
    return mb_size

def get_attribute(filename):
    if filename.endswith('csv'):
        df1 = pd.read_csv(filename,header=[0])
    else:
        df1=pd.read_json(filename,orient="records", lines=True)
    return list(df1.columns) 

def split_large_file(filename, source, output,size=None):
    df1=pd.read_csv(filename,header=[0])
    if size ==None:
        size = get_csv_size(filename)
    num_chunks = size//20
    if num_chunks == 0:
        num_chunks = 1 
    df_all = np.array_split(df1, num_chunks)

    for idx, file in enumerate(df_all):
        file.to_csv('%s\\%s_%03d.csv'%(output, source, idx), index=False)   
        
def show_all_files(folder):
    df = pd.DataFrame(glob.glob('%s/*'%folder), columns = ['path'])
    df['root'] = df.path.apply(lambda x: x.split('\\')[0])
    df['source'] = df.path.apply(lambda x: x.replace('%s\\'%folder,'').split('_')[0])
    df['filename'] = df.path.apply(lambda x: x.replace('%s\\'%folder,'').split('/')[-1])
    df = df.sort_values('source').reset_index(drop=True)
    df['size'] = df['path'].apply(get_csv_size)
    return df

def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


# Check csv

In [86]:
pandarallel.initialize(nb_workers=1, progress_bar=False) 
# Place all files to be filtered within the Filter folder, within a subfolder containing the source name
# for example, let there be 3 csv files from El Espectador. 
# The file structure is 
# Filter
# --elespectador
# -- -- elespactador01.csv
# -- -- elespactador02.csv
# -- -- elespactador03.csv


# This creates a dataframe containing a list of all the csv files within the specified folder structure.
df = pd.DataFrame(glob.glob('Filter\\*\\*.csv'), columns = ['path'])
# This updates the source to be just the first part, the name of the folder.
df['source'] = df.path.apply(lambda x: x.split('\\')[1].replace('.csv', ''))
df = df.sort_values('source').reset_index(drop=True)
# Gets the column defenitions of the files and their sizes
df['attributes']= df.path.apply(get_attribute)
df['size']= df.path.apply(get_csv_size)
df

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,path,source,attributes,size
0,Filter\diaadia\diaadia.csv,diaadia,"[page, rows, link, text]",48.9
1,Filter\elpanam\elpanam.csv,elpanam,"[page, rows, link, text]",2.2
2,Filter\lacritica\lacritica.csv,lacritica,"[page, rows, link, text]",10.1
3,Filter\laprensa\laprensa.csv,laprensa,"[page, rows, link, text]",4.3


Initialize filter files.

In [130]:
# Initialize the relevant keywords and the irrelevant keyword regex's
with open('wiki_relevant_spanish.txt', 'r', encoding='utf8') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

#relevant_rules = re.compile(relevant_rules, re.IGNORECASE)

print('relevant_rules:')
print(relevant_rules)

with open('irrelevant_keywords_spanish.txt', 'r', encoding='utf8') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

#irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

print('\nirelevant_rules:')
print(irelevant_rules)

# debugging, no longer relevant
#find_relevant = lambda x: Counter(relevant_rules.findall(str(x).lower()))
#find_exclude = lambda x: Counter(irelevant_rules.findall(str(x).lower()))

relevant_rules:
\brelaciones \w+-\w+|\bsecuestrar|\babuso|\bactivista|\bactos\b|\badministración|\bcontra las mujeres|\bagencias|\ballegati|\bembajador|\banarquía|\barmado|\bejército|\bdetención|\bartillería|\basesino|\basalto|\basamblea|\basilo|\bataque|\battrocit|\bautoridades|\bboleta electoral|\bbatallas...|\bderramamiento de sangre|\bBLOTTER|\bcuerpos|\bbomba|\bfrontera|\bboicot|\barmarios|\bcaso\b|\bcasualt|\bbajas|\bcensores|\bcivil\b|\bcoaccionar|\bcolonial|\bcombate|\bcomité|\bmancomunidad|\bcomunismo|\bcomunista|\bconflicto|\bcongreso|\bconscripto|\bconservador|\bconstituenc|\bconstitución|\bconvicto|\bcorps\b|\bcorrupto|\bconsejo|\bconcejal|\bContrainsurgencia|\bContra el terrorismo|\bPaíses en|\bgolpes de estado|\btribunales|\bdelitos?|\bcriminal|\brestringir|\bciberataque|\bcibercrimen|\bdaño|\bmuerte|\bmuerte|\bdefensa|\bdefensa|\bdelegado|\bdemocrático|\bdemostración|\bdepartamento|\brepresentación|\bdesplegar|\bdeportar|\bdiputado|\bdestruir|\bdetener|\bdetención|\bdict

In [88]:
# remove all the filtered paths, if they exist.
df['path'] = df['path'].apply(lambda x: x[:-4] + '_filtered.csv')

In [89]:
# Checks for doublely filtered files.
df = df[~df.path.str.contains('_filtered_filtered')]

# Split

In [90]:
# Makes directory for splits. Do not run if a split folder already exists.
cwd = os.getcwd()
os.mkdir(cwd + '\\split\\')
cwd

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\vikas\\OneDrive\\Documents\\GitHub\\ConfliBERT_Filter\\split\\'

In [91]:
# splits all files above a certain MB limit
pandarallel.initialize(nb_workers=8, progress_bar=False)       
_ = df.apply(lambda x: split_large_file(x['path'],x['source'],'split'), axis=1) 

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


FileNotFoundError: [Errno 2] No such file or directory: 'Filter\\diaadia\\diaadia_filtered.csv'

In [92]:
# Debugging, if there are issues here, let me know.
df = show_all_files('split')
df

,path,root,source,filename,size
0,split\diaadia_000.csv,split,diaadia,diaadia_000.csv,28.1
1,split\diaadia_001.csv,split,diaadia,diaadia_001.csv,21.2
2,split\elpanam_000.csv,split,elpanam,elpanam_000.csv,2.3
3,split\lacritica_000.csv,split,lacritica,lacritica_000.csv,10.2
4,split\laprensa_000.csv,split,laprensa,laprensa_000.csv,0.0


In [93]:
# Check file sizes.
df['size'].describe()

count     5.000000
mean     12.360000
std      12.088548
min       0.000000
25%       2.300000
50%      10.200000
75%      21.200000
max      28.100000
Name: size, dtype: float64

# step 1  clean

In [174]:
df = show_all_files('split')
# df['attribute'] = df.path.parallel_apply(get_attribute)
df

,path,root,source,filename,size
0,split\diaadia_000.csv,split,diaadia,diaadia_000.csv,28.1
1,split\diaadia_001.csv,split,diaadia,diaadia_001.csv,21.2
2,split\elpanam_000.csv,split,elpanam,elpanam_000.csv,2.3
3,split\lacritica_000.csv,split,lacritica,lacritica_000.csv,10.2
4,split\laprensa_000.csv,split,laprensa,laprensa_000.csv,0.0


In [95]:
df.source.unique()

array(['diaadia', 'elpanam', 'lacritica', 'laprensa'], dtype=object)

In [175]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

total size: 61.8


,size
source,
diaadia,49.3
elpanam,2.3
lacritica,10.2
laprensa,0.0


In [97]:
# Cleans the given files.
def basic_process(filename, output_folder):
    
    sizes = []
    
    df1 = pd.read_csv(filename, header=[0])
    sizes.append(df1.shape[0])
    
    if 'url' not in df1.columns:
        df1['url'] ='\n'
    if 'title' not in df1.columns:
        df1['title']='\n'
    
#     df1 = df1[~df1.url.isnull()]
    
    df1 = df1[~df1.text.isnull()]
    df1 = df1[~df1.text.duplicated()]
#     df1 = df1[~df1.url.duplicated()]
    
    df1.loc[df1.title.isnull(),'title']='\n'
    df1.loc[df1.url.isnull(),'url']='\n'
    if 'abstract' in df1:
        df1.loc[df1.abstract.isnull(),'abstract']='\n'
        
    # ------------------- Start cleaning  --------------------------#
    
    # Convert coding
    df1.text = df1.text.apply(lambda x: unicodetoascii(x))
    df1.text = df1.text.apply(lambda x: unicodedata.normalize("NFKD", x))
    
    # email
    df1.text = df1.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))
    
    # telphone
    df1.text = df1.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
    df1.text = df1.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))
    
    # noise
    df1.text =\
    df1.text.apply(lambda x: re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE))
    
    # urls
    df1.text = df1.text.apply(lambda x: re.sub(r"http\S+", "", x))
    
    # delete too many \n
    df1.text = df1.text.apply(lambda x: re.sub('\n\n+', "\n\n", x, flags=re.IGNORECASE))
    
    # head and tails
    df1.text = df1.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)) 
    
    
    df1 = df1[df1.text.str.len()>100]
    
    sizes.append(df1.shape[0])
    
    # ------------------- Ending cleaning  --------------------------#
    
    filename = filename.split('\\')[1]
    new_filename = output_folder + '/'+ filename

    print('%s:\t%s'%(filename, sizes))
    df1.to_csv(new_filename, index= False)
    return

In [176]:
# Takes the splits and filters them.
pandarallel.initialize(nb_workers=8, progress_bar=False) 
_ = df['path'].apply(basic_process, output_folder='split')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
diaadia_000.csv:	[10741, 10741]
diaadia_001.csv:	[10767, 10767]
elpanam_000.csv:	[619, 619]
lacritica_000.csv:	[2959, 2959]
laprensa_000.csv:	[1, 1]


In [177]:
df = show_all_files('split')
df

,path,root,source,filename,size
0,split\diaadia_000.csv,split,diaadia,diaadia_000.csv,28.1
1,split\diaadia_001.csv,split,diaadia,diaadia_001.csv,21.2
2,split\elpanam_000.csv,split,elpanam,elpanam_000.csv,2.3
3,split\lacritica_000.csv,split,lacritica,lacritica_000.csv,10.2
4,split\laprensa_000.csv,split,laprensa,laprensa_000.csv,0.0


In [178]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

total size: 61.8


,size
source,
diaadia,49.3
elpanam,2.3
lacritica,10.2
laprensa,0.0


### Filter

In [179]:
# defines a metric for number of relevant and irrelevant words.
def find_relevant(str):
    return len(re.findall(relevant_rules, str))
def find_irelevant(str):
    return len(re.findall(irelevant_rules, str))

In [180]:
# find_relevant = lambda x: Counter(relevant_rules.findall(x.lower()))
# find_exclude = lambda x: Counter(irelevant_rules.findall(x.lower()))
#

# for each file, open and read into dataframe, then make sure text is NOT null
for path in df.path:
    df1 = pd.read_csv(path, header=[0])
    df1 = df1[~df1.text.isnull()]
    df1['relevant'] = df1['text'].apply(find_relevant)
    df1['irrelevant'] = df1['text'].apply(find_irelevant)
# The next three lines are the filter, can be changed based on the needs of the project. Currently, throw out all rows with more than two irrelevant words/regex
    df2 = df1[(df1['relevant'] < 2)]
    df2 = df2[(df2['irrelevant'] > 5)]
    df1 = pd.merge(df1,df2, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
    
    df1 = df1.drop('relevant', axis=1)
    df1 = df1.drop('irrelevant', axis=1)
# next line is debugging    
    # print(df1)
# create a filtered file with the given dataframe, will appear in same folder as unfiltered

    df1.to_csv(path[:-4] + '_filtered.csv')

# save to json

In [181]:
# folder = 'none'
# df = pd.DataFrame(columns=['path','source', 'filename'])
# df.path = [str(x) for x in pathlib.Path('%s/*'%folder).glob('**/*')]
df = pd.DataFrame(glob.glob('*\\*'), columns = ['path'])
df = df[df.path.str.contains('_filtered')]
# df['root'] = df.path.apply(lambda x: x.split('/')[0])

df['source'] = df.path.apply(lambda x: x.split('\\')[1][0:x.index('_')-6])
df['filename'] = df.path.apply(lambda x: x.split('\\')[-1])

# df = df.sort_values('source').reset_index(drop=True)
# df['size'] = df['path'].parallel_apply(get_csv_size)
# df['filename'] = df.path.apply(lambda x: x.replace('%s/'%folder,''))

df['json_file'] = df['filename'].apply(lambda x: x.replace('.csv', '.json'))

df['json_file'] = df.source +'\\'+ df.json_file
df['tar_file'] = 'tar\\'+ df.json_file+'.tar.gz'
df.json_file = 'json\\'+ df.json_file
df

,path,source,filename,json_file,tar_file
9,split\diaadia_000_filtered.csv,diaadia,diaadia_000_filtered.csv,json\diaadia\diaadia_000_filtered.json,tar\diaadia\diaadia_000_filtered.json.tar.gz
11,split\diaadia_001_filtered.csv,diaadia,diaadia_001_filtered.csv,json\diaadia\diaadia_001_filtered.json,tar\diaadia\diaadia_001_filtered.json.tar.gz
13,split\elpanam_000_filtered.csv,elpanam,elpanam_000_filtered.csv,json\elpanam\elpanam_000_filtered.json,tar\elpanam\elpanam_000_filtered.json.tar.gz
15,split\lacritica_000_filtered.csv,lacritica,lacritica_000_filtered.csv,json\lacritica\lacritica_000_filtered.json,tar\lacritica\lacritica_000_filtered.json.tar.gz
17,split\laprensa_000_filtered.csv,laprensa,laprensa_000_filtered.csv,json\laprensa\laprensa_000_filtered.json,tar\laprensa\laprensa_000_filtered.json.tar.gz


In [154]:
os.mkdir(cwd + '\\json\\')
os.mkdir(cwd + '\\tar\\')

for i in df.source.unique():
    os.mkdir(cwd + '\\json\\'+i)
    os.mkdir(cwd + '\\tar\\'+i)  

In [156]:
df1 = pd.read_csv(df.path[5], header=[0])
df1

,Unnamed: 0,page,rows,link,text,url,title
0,0,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/casi-el-40-de-la-poblacion-evita-las-noticias-porque-deterioran-su-animo-tik-tok-le-gana"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/comunicacion.jpg?itok=-O1cQD0J"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/casi-el-40-de-la-poblacion-evita-las-noticias-porque-deterioran-su-animo-tik-tok-le-gana"">\n<h5 class=""tit...",https://www.diaadia.com.pa/mundo/casi-el-40-de-la-poblacion-evita-las-noticias-porque-deterioran-su-animo-tik-tok-le-gana,"Buena parte de la población mundial -un 38 %- evita con frecuencia consumir noticias de actualidad como las relacionadas con la guerra en Ucrania o la pandemia pues ""deterioran"" su estado de ánimo, según un informe difundido este miércoles por el Instituto Reuters, que desvela que los más jóvenes prefieren plataformas como TikTok.\n\nEl estudio sobre consumo de noticias Digital News Report 2022, presentado hoy en la sede en Londres de la agencia de noticias británica, se basa en una encuesta online a 93.000 personas en 46 mercados, entre ellos España, Argentina, Brasil, Chile, Colo...",\n,\n
1,1,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/donan-500000-euros-acnur-para-ayuda-humanitaria-en-ucrania-751758"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/Ucrania%20DD.jpg?itok=laMENc2k"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/donan-500000-euros-acnur-para-ayuda-humanitaria-en-ucrania-751758"">\n<h5 class=""title-fixed der"">Donan 500.000 euros a ACNUR para...",https://www.diaadia.com.pa/mundo/donan-500000-euros-acnur-para-ayuda-humanitaria-en-ucrania-751758,"Brother Industries, Ltd. anunció que el grupo donará 500.000 euros a ACNUR (Alto Comisionado de las Naciones Unidas para los Refugiados).\n\nEsta donación se entregará a través de ACNUR para apoyar los esfuerzos de ayuda humanitaria en Ucrania y los países vecinos.\n\nEl corporativo (proveedor en tecnología de equipos de oficina) lamenta enormemente que tantas vidas inocentes se vean afectadas negativamente por la situación actual en Europa del Este y ha decidido extender su apoyo a los necesitados.\n\nEsperan que todos los afectados estén a salvo y agregaron que sus pensamientos ...",\n,\n
2,2,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/un-farmaco-elimina-el-cancer-rectal-en-los-12-pacientes-de-un-ensayo-clinico-751565"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/cancer%20.jpg?itok=pmXy05Yy"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/un-farmaco-elimina-el-cancer-rectal-en-los-12-pacientes-de-un-ensayo-clinico-751565"">\n<h5 class=""title-fixed der""...",https://www.diaadia.com.pa/mundo/un-farmaco-elimina-el-cancer-rectal-en-los-12-pacientes-de-un-ensayo-clinico-751565,"Los doce pacientes con cáncer rectal que participaron en un ensayo clínico en el que durante al menos seis meses recibieron Dostarlimab han superado el cáncer sin necesidad de someterse después a un tratamiento de quimioterapia o a una cirugía.\n\nLos resultados del ensayo se han publicado en la revista New England Journal of Medicine y han sido patrocinados por la farmacéutica

In [182]:
def convert_json_tar(filename, json_file, tar_file):
    df1 = pd.read_csv(filename, header=[0])      
    df1.to_json(json_file, orient="records", lines=True)
    
    with tarfile.open(tar_file, "w:gz") as tar:
        tar.add(json_file, arcname=os.path.basename(json_file))
df.apply(lambda x: convert_json_tar(x['path'], x['json_file'], x['tar_file']), axis=1)

9     None
11    None
13    None
15    None
17    None
dtype: object

In [183]:
pandarallel.initialize() 

df.source = df.path.apply(lambda x: x.split('\\')[1][0:x.index('_')-6]).values
df['size'] = df['path'].apply(get_csv_size)
df

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,path,source,filename,json_file,tar_file,size
9,split\diaadia_000_filtered.csv,diaadia,diaadia_000_filtered.csv,json\diaadia\diaadia_000_filtered.json,tar\diaadia\diaadia_000_filtered.json.tar.gz,26.9
11,split\diaadia_001_filtered.csv,diaadia,diaadia_001_filtered.csv,json\diaadia\diaadia_001_filtered.json,tar\diaadia\diaadia_001_filtered.json.tar.gz,20.5
13,split\elpanam_000_filtered.csv,elpanam,elpanam_000_filtered.csv,json\elpanam\elpanam_000_filtered.json,tar\elpanam\elpanam_000_filtered.json.tar.gz,2.3
15,split\lacritica_000_filtered.csv,lacritica,lacritica_000_filtered.csv,json\lacritica\lacritica_000_filtered.json,tar\lacritica\lacritica_000_filtered.json.tar.gz,9.9
17,split\laprensa_000_filtered.csv,laprensa,laprensa_000_filtered.csv,json\laprensa\laprensa_000_filtered.json,tar\laprensa\laprensa_000_filtered.json.tar.gz,0.0


ValueError: '_' is not in list

In [186]:
print(df['size'].sum())

df.groupby(by=["source"], dropna=False).sum()

59.599999999999994


,size
source,
diaadia,47.4
elpanam,2.3
lacritica,9.9
laprensa,0.0
